In [1]:
from decouple import config
from qdrant_client import QdrantClient


QDRANT_URL = config('QDRANT_URL')
QDRANT_API_KEY = config('QDRANT_API_KEY')

In [ ]:
from .utils import load_datasets


corpus_df, queries_df, qrels_df = load_datasets(10_000, 10_000)

In [2]:
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

collections=[]


In [ ]:
from fastembed import (
    TextEmbedding, 
    SparseTextEmbedding, 
    LateInteractionTextEmbedding
)
from qdrant_client.models import (
    Distance, 
    VectorParams, 
    SparseVectorParams, 
    MultiVectorConfig,
    MultiVectorComparator,
    Modifier
)

from .models.config import (
    DenseModelConfig, 
    SparseModelConfig,
    RerankingModelConfig
)


dense_model_name = 'BAAI/bge-small-en-v1.5'
dense_model = TextEmbedding(model_name=dense_model_name)

sparse_model_name = 'Qdrant/bm25'
sparse_model = SparseTextEmbedding(
    model_name=sparse_model_name,
    k1=1.5,
    b=0.75
)

reranking_model_name = 'colbert-ir/colbertv2.0'
reranking_model = LateInteractionTextEmbedding(reranking_model_name)

dense_model_config = DenseModelConfig(
    name=dense_model_name,
    vector_params=VectorParams(
        size=384,
        distance=Distance.COSINE,
    )
)

sparse_model_config = SparseModelConfig(
    name=sparse_model_name,
    sparse_vector_params=SparseVectorParams(modifier=Modifier.IDF)
)

reranking_model_config = RerankingModelConfig(
    name=reranking_model_name,
    vector_params=VectorParams(
        size=100,
        distance=Distance.COSINE,
        multivector_config=MultiVectorConfig(
            comparator=MultiVectorComparator.MAX_SIM,
        )
    )
)

In [ ]:
documents = [
    'FastEmbed is lighter than Transformers & Sentence-Transformers.',
    'FastEmbed is supported by and maintained by Qdrant.',
]

dense_embeddings = list(dense_model.embed(documents))
sparse_embeddings = list(sparse_model.embed(documents))
reranking_embeddings = list(reranking_model.embed(documents))

In [ ]:
from .models import HybridSearch, HybridRerankingSearch


COLLECTION_NAME = 'collection_1'

hybrid_search = HybridSearch(qdrant_client, dense_model_config, sparse_model_config)
hybrid_search.create_collection(COLLECTION_NAME)
hybrid_search.upsert(COLLECTION_NAME, ...)  # TODO prepare data

In [ ]:
import json
from fastembed.sparse import SparseEmbedding
from tokenizers import Tokenizer


def get_tokens_and_weights(sparse_embedding: SparseEmbedding, tokenizer: Tokenizer) -> dict:
    token_weight_dict = {}
    
    for i in range(len(sparse_embedding.indices)):
        token = tokenizer.decode([sparse_embedding.indices[i]])
        weight = sparse_embedding.values[i]
        token_weight_dict[token] = weight

    # sort by weights
    token_weight_dict = dict(sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True))
    
    return token_weight_dict


tokenizer = Tokenizer.from_pretrained('Qdrant/SPLADE_PP_en_v1')
index = 0
print(json.dumps(get_tokens_and_weights(sparse_embeddings[index], tokenizer), indent=4))

In [ ]:
# TODO add nvidia API for LLM